<a href="https://colab.research.google.com/github/lordoz234/ordoz234-Tabular-Playground-Series---Oct-2021/blob/main/AutoGluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogluon.tabular[all]

     |████████████████████████████████| 273 kB 5.1 MB/s 
     |████████████████████████████████| 27.4 MB 98 kB/s 
     |████████████████████████████████| 296 kB 63.4 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 22.3 MB 79.6 MB/s 
     |████████████████████████████████| 352 kB 59.5 MB/s 
     |████████████████████████████████| 2.0 MB 55.3 MB/s 
     |████████████████████████████████| 67.3 MB 6.4 kB/s 
     |████████████████████████████████| 186 kB 58.7 MB/s 
     |████████████████████████████████| 166.7 MB 19 kB/s 
     |████████████████████████████████| 4.2 MB 61.0 MB/s 
     |████████████████████████████████| 206 kB 69.5 MB/s 
     |████████████████████████████████| 131 kB 71.8 MB/s 
     |████████████████████████████████| 786 kB 70.7 MB/s 
     |████████████████████████████████| 1.0 MB 47.2 MB/s 
     |████████████████████████████████| 125 kB 51.7 MB/s 
     |████████████████████████████████| 56 kB 3.6 MB/s 
     |█████████████

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
train_data = pd.read_csv("/content/drive/My Drive/X5-MIPT/tabular-playground-series-oct-2021/train.csv")
test_data = pd.read_csv("/content/drive/My Drive/X5-MIPT/tabular-playground-series-oct-2021/test.csv")
submission = pd.read_csv("/content/drive/My Drive/X5-MIPT/tabular-playground-series-oct-2021/sample_submission.csv")

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 287 entries, id to target
dtypes: float64(240), int64(47)
memory usage: 2.1 GB


In [6]:
features = [col for col in train_data.columns if 'f' in col]

cont_features =[]
disc_features =[]

for col in features:
    if train_data[col].dtype=='float64':
        cont_features.append(col)
    else:
        disc_features.append(col)
        
train_data[cont_features] = train_data[cont_features].astype('float32')
train_data[disc_features] = train_data[disc_features].astype('uint8')
train_data[cont_features] = train_data[cont_features].astype('float32')
train_data[disc_features] = train_data[disc_features].astype('uint8')

In [ ]:
train_data.info()

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(10)


In [9]:
from sklearn.model_selection import train_test_split

t0 = timer()
X_train, X_val = train_test_split(train_data, test_size = 0.2)
t1 = timer()

In [10]:
print("Train_test_split time: {} seconds".format(t1 - t0))

Train_test_split time: 2.257747643000016 seconds


In [11]:
from  autogluon.tabular import TabularPredictor

random_state = 55
label = 'target'

hyperparameters = {
    'GBM': [
        {'extra_trees': True, 'seed': random_state, 'ag_args': {'name_suffix': 'XT'}},
        {},
    ],
    'RF': [
        {'criterion': 'gini', 'random_state': random_state,
         'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary']}
         },
    ],
    'KNN': [
        {'weights': 'uniform', 'n_neighbors': 1000, 'ag_args': {'name_suffix': 'Unif'}
         },
        {'weights': 'distance', 'n_neighbors': 1000, 'ag_args': {'name_suffix': 'Dist'}
         },
    ],
}

t0 = timer()

autogluon_predictor = TabularPredictor(
    label = label,
    eval_metric = "roc_auc",
    learner_kwargs={'ignored_columns': ['id']}
).fit(
    train_data=X_train,
    verbosity = 4, 
    hyperparameters=hyperparameters
)

t1 = timer()
fitting_time = t1 - t0

No path specified. Models will be saved in: "AutogluonModels/ag-20211010_175549/"
============ fit kwarg info ============
User Specified kwargs:
{'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "Autogluon

[1]	train_set's binary_logloss: 0.678793	valid_set's binary_logloss: 0.678647
[2]	train_set's binary_logloss: 0.665625	valid_set's binary_logloss: 0.665274
[3]	train_set's binary_logloss: 0.65386	valid_set's binary_logloss: 0.653347
[4]	train_set's binary_logloss: 0.642797	valid_set's binary_logloss: 0.642167
[5]	train_set's binary_logloss: 0.632667	valid_set's binary_logloss: 0.63197
[6]	train_set's binary_logloss: 0.623336	valid_set's binary_logloss: 0.622463
[7]	train_set's binary_logloss: 0.614956	valid_set's binary_logloss: 0.613958
[8]	train_set's binary_logloss: 0.60735	valid_set's binary_logloss: 0.606262
[9]	train_set's binary_logloss: 0.600398	valid_set's binary_logloss: 0.599228
[10]	train_set's binary_logloss: 0.594016	valid_set's binary_logloss: 0.592766
[11]	train_set's binary_logloss: 0.58817	valid_set's binary_logloss: 0.586837
[12]	train_set's binary_logloss: 0.58277	valid_set's binary_logloss: 0.581344
[13]	train_set's binary_logloss: 0.577733	valid_set's binary_loglo

Saving AutogluonModels/ag-20211010_175549/models/LightGBMXT/model.pkl
	0.8538	 = Validation score   (roc_auc)
	1790.91s	 = Training   runtime
	0.85s	 = Validation runtime
Saving AutogluonModels/ag-20211010_175549/models/trainer.pkl
Fitting model: LightGBM ...
	Dropped 0 of 285 features.
	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 2
Training Gradient Boosting Model for 10000 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'learning_rate': 0.05, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True}
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	train_set's binary_logloss: 0.677525	valid_set's binary_logloss: 0.677522
[2]	train_set's binary_logloss: 0.663403	valid_set's binary_logloss: 0.663411
[3]	train_set's binary_logloss: 0.65061	valid_set's binary_logloss: 0.650548
[4]	train_set's binary_logloss: 0.638999	valid_set's binary_logloss: 0.638881
[5]	train_set's binary_logloss: 0.628408	valid_set's binary_logloss: 0.628245
[6]	train_set's binary_logloss: 0.618758	valid_set's binary_logloss: 0.61858
[7]	train_set's binary_logloss: 0.609945	valid_set's binary_logloss: 0.609687
[8]	train_set's binary_logloss: 0.601871	valid_set's binary_logloss: 0.601543
[9]	train_set's binary_logloss: 0.594473	valid_set's binary_logloss: 0.59415
[10]	train_set's binary_logloss: 0.587693	valid_set's binary_logloss: 0.58732
[11]	train_set's binary_logloss: 0.581453	valid_set's binary_logloss: 0.581042
[12]	train_set's binary_logloss: 0.575744	valid_set's binary_logloss: 0.575304
[13]	train_set's binary_logloss: 0.570482	valid_set's binary_logl

Saving AutogluonModels/ag-20211010_175549/models/LightGBM/model.pkl
	0.8541	 = Validation score   (roc_auc)
	1370.48s	 = Training   runtime
	0.61s	 = Validation runtime
Saving AutogluonModels/ag-20211010_175549/models/trainer.pkl
Fitting model: RandomForestGini ...
	Dropped 0 of 285 features.
	Fitting RandomForestGini with 'num_gpus': 0, 'num_cpus': 2
Saving AutogluonModels/ag-20211010_175549/models/RandomForestGini/model.pkl
	0.8295	 = Validation score   (roc_auc)
	2120.6s	 = Training   runtime
	0.74s	 = Validation runtime
Saving AutogluonModels/ag-20211010_175549/models/trainer.pkl
Loading: AutogluonModels/ag-20211010_175549/models/LightGBM/model.pkl
Loading: AutogluonModels/ag-20211010_175549/models/LightGBMXT/model.pkl
Loading: AutogluonModels/ag-20211010_175549/models/RandomForestGini/model.pkl
Model configs that will be trained (in order):
	WeightedEnsemble_L2: 	{'ag_args': {'valid_base': False, 'name_bag_suffix': '', 'model_type': <class 'autogluon.core.models.greedy_ensemble.gr

In [ ]:
print("Fitting time: {} seconds".format(fitting_time))

In [ ]:
t0 = timer()
leaderboard = autogluon_predictor.leaderboard(X_val)
t1 = timer()

In [ ]:
print("Getting leaderboard time: {} seconds".format(t1 - t0))

In [ ]:
t0 = timer()
predictions = autogluon_predictor.predict_proba(test_data)
t1 = timer()
prediction_time = t1 - t0

In [16]:
print("Predictions time: {} seconds".format(prediction_time))

Predictions time: 93.56848292500035 seconds
Predictions time: 93.56848292500035 seconds


In [17]:
submission.target = predictions.iloc[:, 1]

In [18]:
submission

,id,target
0,1000000,0.766663
1,1000001,0.242460
2,1000002,0.902120
3,1000003,0.836872
4,1000004,0.274387
...,...,...
499995,1499995,0.976079
499996,1499996,0.801931
499997,1499997,0.304867
499998,1499998,0.567129


In [19]:
submission.to_csv("submit.csv", index = False)